In [1]:
import numpy as np
import torch
import torch.nn as nn

In [88]:
def convtranspose2d(input_data, weight, bias=None, stride=1, padding=0, output_padding=0):
    # Определяем размеры входных данных
    input_height, input_width = input_data.shape
    output_height, output_width = (input_height-1)*stride - 2*padding + weight.shape[2] + output_padding, (input_width-1)*stride - 2*padding + weight.shape[3] + output_padding
    
    # Создаем выходной массив
    output = np.zeros((output_height, output_width))
    
    # Добавляем padding
    padded_input = np.pad(input_data, ((padding, padding), (padding, padding)), mode='constant')
    
    # Производим транспонированную свертку
    for i in range(0, input_height):
        for j in range(0, input_width):
            output[i*stride:i*stride+weight.shape[2], j*stride:j*stride+weight.shape[3]] += weight[0, 0] * input_data[i, j]
    
    # Добавляем bias, если он указан
    if bias is not None:
        output += bias
    
    return output

In [105]:
def test_convtranspose2d(input_data, weight, bias):
    stride = 2
    padding = 1
    output_padding = 1

    output_your_function = convtranspose2d(input_data, weight, bias, stride, padding, output_padding)
    
    input_tensor = torch.from_numpy(input_data).unsqueeze(0).unsqueeze(0)  # Преобразование входных данных в тензор PyTorch
    weight_tensor = torch.from_numpy(weight).permute(3, 2, 0, 1)  # Преобразование весов в тензор PyTorch
    bias_tensor = torch.from_numpy(bias) if bias is not None else None  # Преобразование bias в тензор PyTorch

    conv_transpose = torch.nn.ConvTranspose2d(1, 16, 3, stride=stride, padding=padding, output_padding=output_padding)
    conv_transpose.weight.data = weight_tensor
    if bias is not None:
        conv_transpose.bias.data = bias_tensor
    
    output_pytorch = conv_transpose(input_tensor).squeeze().detach().numpy()  # Результат работы pytorch.CONVTRANSPOSE2D

    assert np.allclose(output_your_function, output_pytorch, rtol=1e-4, atol=1e-4), "Тест провален!"
    
    print("Тест пройден успешно!")

In [108]:
test_convtranspose2d(np.random.rand(28, 28), np.random.rand(16, 16, 3, 3), np.random.rand(16, 16))

Тест пройден успешно!


In [109]:
test_convtranspose2d(np.random.rand(32, 32), np.random.rand(8, 8, 3, 3), np.random.rand(8, 8))

Тест пройден успешно!


In [112]:
test_convtranspose2d(np.random.rand(20, 20), np.random.rand(5, 5, 3, 3), bias = None)

Тест пройден успешно!


## Доп. задание

In [98]:
class MyConvTranspose2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.output_padding = output_padding
        self.weight = np.random.rand(out_channels, in_channels, kernel_size, kernel_size)
    
    def forward(self, input):
        batch_size, in_channels, input_height, input_width = input.shape
        
        # Calculate output size
        output_height = (input_height - 1) * self.stride - 2*self.padding + self.kernel_size + self.output_padding
        output_width = (input_width - 1) * self.stride - 2*self.padding + self.kernel_size + self.output_padding
        
        output = np.zeros((batch_size, self.out_channels, output_height, output_width))
        
        for b in range(batch_size):
            for oc in range(self.out_channels):
                for ic in range(self.in_channels):
                    for i in range(output_height):
                        for j in range(output_width):
                            for k in range(self.kernel_size):
                                for l in range(self.kernel_size):
                                    x = i + k - self.padding
                                    y = j + l - self.padding
                                    if x % self.stride == 0 and y % self.stride == 0 and x >= 0 and x < input_height*self.stride and y >=0 and y < input_width*self.stride:
                                        output[b, oc, i, j] += input[b, ic, x // self.stride, y // self.stride] * self.weight[oc, ic, k, l]
        
        return output

In [99]:
def test_convT(in_channels, out_channels, kernel_size, stride, padding):
    # Создаем случайные входные данные
    input_data = np.random.rand(1, 3, 4, 4)
    input_torch = torch.tensor(input_data)
    input_np = np.transpose(input_data, (0, 2, 3, 1))  # меняем порядок размерностей для компатимости с PyTorch
    
    # Создаем экземпляр нашего класса и класса из PyTorch
    convT_custom = MyConvTranspose2D(in_channels, out_channels, kernel_size, stride, padding)
    convT_torch = nn.ConvTranspose2d(n_channels, out_channels, kernel_size, stride=stride, padding=padding)
    
    # Копируем веса из PyTorch в наш класс
    convT_custom.weight = convT_torch.weight.detach().numpy()
    
    # Выполняем транспонированную свертку с помощью обоих классов
    output_custom = convT_custom.forward(input_np)
    output_custom_torch = convT_torch(input_torch).detach().numpy()
    
    # Сравниваем результаты
    assert np.allclose(output_custom, output_custom_torch, rtol=1e-5), "Тест провален!"
    
    print("Тест пройден успешно!")

In [102]:
test_convT(3, 2, 3, 1, 1)

Тест пройден успешно!


In [103]:
test_convT(6, 32, 2, 1, 3)

Тест пройден успешно!


In [104]:
test_convT(4, 16, 4, 2, 4)

Тест пройден успешно!
